Set the filename

In [1]:
filename = "/home/gregorylemieux/Work/Issues/383/deciduous-griddedruns/cesm.log.1805694.chadmin1.ib0.cheyenne.ucar.edu.200417-103849"

"/home/gregorylemieux/Work/Issues/383/deciduous-griddedruns/cesm.log.1805694.chadmin1.ib0.cheyenne.ucar.edu.200417-103849"

In [2]:
# Open the file
f = open(filename)

# Read through each line and pull out the unique threadnumbers
A = Array{Int64}(undef,0) # Initialize
for line in eachline(f)
    threadnum = parse(Float64,split(line,":")[1]) # Parse the line for the thread number
    if !(in(threadnum,A)) # If not in the array add it
        push!(A,threadnum)
    end
end

# Close the file
close(f)

Run a couple of checks

In [3]:
# Check all entries are unique
println(A == unique(A))

# Check that the thread numbers are sequential
println(sort!(A) == Array(0:maximum(A)))

true
true


Run through the file again and count the number of times that the laimemory is zero

In [172]:
# Set the known strings we are interested in
string_canopy = "Starting canopy trim"
string_cohort = "Current cohort"
string_laimem = "Starting laimemory"
string_season = "EDPftvarcon_inst%season_decid"
string_stress = "EDPftvarcon_inst%stress_decid"
string_evergreen = "EDPftvarcon_inst%evergreen"

# Create a dictionary to store the information
checkd = Dict("season"=>0,"stress"=>0,"evergreen"=>0)

Dict{String,Int64} with 3 entries:
  "stress"    => 0
  "season"    => 0
  "evergreen" => 0

In [173]:
for t in A[2]
    println(t)
    totallaicount = 0
    totallinecount = 0
    totalmatchcount = 0
    
    laiflag = 0
    
    f = open(filename) # Open the file`
    
    # Iterate through file
    for line in eachline(f)
        totallinecount += 1
        
        linesplit = split(line,":") # Split into array of strings
        threadnum = parse(Float64,linesplit[1]) # Parse the line for the thread number
        
        # Make sure the line format is threadnum:string:val
        if threadnum == t && length(linesplit) == 3 && isempty(findall(isempty,linesplit))
            if strip(linesplit[2]) == string_laimem
                totallaicount += 1  # Count up the total laimemory loops
                if parse(Float64,strip(linesplit[3])) == 0 # Set the laiflag for the next read
                    laiflag = 1
                else
                    laiflag = 0
                end
            elseif strip(linesplit[2]) == string_season && laiflag == 1
                checkd["season"] += 1
                laiflag = 0
            elseif strip(linesplit[2]) == string_stress && laiflag == 1
                checkd["stress"] += 1
                laiflag = 0
            elseif strip(linesplit[2]) == string_evergreen && laiflag == 1
                checkd["evergreen"] += 1
                laiflag = 0
            end
        end
    end
close(f) # Close the file
end

1


In [174]:
checkd

Dict{String,Int64} with 3 entries:
  "stress"    => 0
  "season"    => 7456
  "evergreen" => 0

In [ ]:
l